In [1]:
import pandas as pd
import numpy as np

In [3]:
import os
import pandas as pd

label = []
for file_name in os.listdir('audios/train'):
    label.append(f'audios/train/{file_name}')

# Correct way to create DataFrame
df_file = pd.DataFrame(label, columns=['filename'])

In [4]:
df_file.head()

,filename
0,audios/train/audio_1.wav
1,audios/train/audio_10.wav
2,audios/train/audio_100.wav
3,audios/train/audio_101.wav
4,audios/train/audio_102.wav


In [7]:
df_file.shape, train.shape

((409, 1), (409, 2))

In [6]:
train = pd.read_csv('csvs/train.csv')
train.head()

,filename,label
0,audio_173,3.0
1,audio_138,3.0
2,audio_127,2.0
3,audio_95,2.0
4,audio_73,3.5


In [ ]:
import pandas as pd

df_file['filename_clean'] = df_file['filename'].apply(lambda x: x.split('/')[-1].replace('.wav', ''))

df_merged = pd.merge(df_file, train, left_on='filename_clean', right_on='filename', how='left')

df_merged.drop(columns=['filename_clean'], inplace=True)

In [12]:
df_merged.head()

,filename_x,filename_y,label
0,audios/train/audio_1.wav,audio_1,3.0
1,audios/train/audio_10.wav,audio_10,3.0
2,audios/train/audio_100.wav,audio_100,3.0
3,audios/train/audio_101.wav,audio_101,3.5
4,audios/train/audio_102.wav,audio_102,3.0


In [13]:
df = df_merged.drop(columns='filename_y')

In [14]:
df

,filename_x,label
0,audios/train/audio_1.wav,3.0
1,audios/train/audio_10.wav,3.0
2,audios/train/audio_100.wav,3.0
3,audios/train/audio_101.wav,3.5
4,audios/train/audio_102.wav,3.0
...,...,...
404,audios/train/audio_95.wav,2.0
405,audios/train/audio_96.wav,3.0
406,audios/train/audio_97.wav,2.0
407,audios/train/audio_98.wav,3.0


In [22]:
import librosa

y, sr = librosa.load('audios/train/audio_1.wav', mono=False)
print(f"Waveform shape: {y.shape}")
print(f"Sample rate: {sr}")

C:\Users\piyus\AppData\Local\Temp\ipykernel_12724\1773321686.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load('audios/train/audio_1.wav', mono=False)


NoBackendError: 

In [24]:
import soundfile as sf
import os
from tqdm import tqdm
AUDIO_ROOT = "audios"
def validate_audio_files(audio_root, mode, filenames):
    """Validate all audio files and return lists of valid and corrupted files"""
    valid_files = []
    corrupted_files = []
    
    for fn in tqdm(filenames, desc=f"Validating {mode} files"):
        file_path = f"{audio_root}/{mode}/{fn}.wav"
        try:
            # Try to read file metadata without loading full audio
            info = sf.info(file_path)
            # Additional check: try to read first few samples
            data, sr = sf.read(file_path, frames=1000)
            valid_files.append(fn)
        except Exception as e:
            print(f"Corrupted: {fn} - {e}")
            corrupted_files.append(fn)
    
    return valid_files, corrupted_files

# Validate before processing
print("Validating audio files...")
train_valid, train_corrupted = validate_audio_files(
    AUDIO_ROOT, "train", train_df["filename"].tolist()
)
test_valid, test_corrupted = validate_audio_files(
    AUDIO_ROOT, "test", test_df["filename"].tolist()
)

print(f"\nTrain: {len(train_valid)} valid, {len(train_corrupted)} corrupted")
print(f"Test: {len(test_valid)} valid, {len(test_corrupted)} corrupted")

# Filter dataframes to keep only valid files
train_df = train_df[train_df["filename"].isin(train_valid)].reset_index(drop=True)
test_df = test_df[test_df["filename"].isin(test_valid)].reset_index(drop=True)

# Save list of corrupted files
with open('corrupted_files.txt', 'w') as f:
    f.write("TRAIN CORRUPTED:\n")
    f.write("\n".join(train_corrupted))
    f.write("\n\nTEST CORRUPTED:\n")
    f.write("\n".join(test_corrupted))
 

Validating audio files...


NameError: name 'train_df' is not defined

In [2]:
import whisper

model = whisper.load_model("base", device="cuda")

result = model.transcribe(
    r"C:\Users\piyus\Downloads\shl-intern-hiring-assessment-2025\audios\train\audio_1.wav"
)

print(result["text"])


FileNotFoundError: [WinError 2] The system cannot find the file specified